# Introduction to machine learning: classification of basalt source assignment

## Import scientific python libraries

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import copy

## Machine learning
Text from: https://scikit-learn.org/stable/tutorial/basic/tutorial.html

In general, a learning problem considers a set of n samples of data and then tries to predict properties of unknown data. If each sample is more than a single number and, for instance, a multi-dimensional entry (aka multivariate data), it is said to have several attributes or features.

Learning problems fall into a few categories:
- **supervised learning**, in which the data comes with additional attributes that we want to predict (Click here to go to the scikit-learn supervised learning page).This problem can be either:
    - *classification*: samples belong to two or more classes and we want to learn from already labeled data how to predict the class of unlabeled data. An example of a classification problem would be handwritten digit recognition, in which the aim is to assign each input vector to one of a finite number of discrete categories. Another way to think of classification is as a discrete (as opposed to continuous) form of supervised learning where one has a limited number of categories and for each of the n samples provided, one is to try to label them with the correct category or class.
    - *regression*: if the desired output consists of one or more continuous variables, then the task is called regression. An example of a regression problem would be the prediction of the length of a salmon as a function of its age and weight.

- **unsupervised learning**, in which the training data consists of a set of input vectors x without any corresponding target values. The goal in such problems may be to discover groups of similar examples within the data, where it is called clustering, or to determine the distribution of data within the input space, known as density estimation, or to project the data from a high-dimensional space down to two or three dimensions for the purpose of visualization (Click here to go to the Scikit-Learn unsupervised learning page).

### Training set and testing set

Machine learning is about learning some properties of a data set and then testing those properties against another data set. A common practice in machine learning is to evaluate an algorithm by splitting a data set into two. We call one of those sets the training set, on which we learn some properties; we call the other set the testing set, on which we test the learned properties.

**Today we will focus on classification through a supervised learning approach**

*Systems doing this type of analysis are all around us. Consider a spam filter for example*

# Classifying volcanic rocks

<img src="./images/volcanic-tectonics.png" width = 600 align = 'center'>

Today we are going to continue dealing with igneous geochemistry data. Igneous rocks are those that crystallize from cooling magma. Different magmas have different compositions associated with their origin as we explore two weeks ago. During class today, we will continue to focus on data from mafic lava flows (these are called basalts and are the relatively low silica, high iron end of what we looked at two weeks ago).

> Igneous rocks form in a wide variety of tectonic settings,
including mid-ocean ridges, ocean islands, and volcanic
arcs. It is a problem of great interest to igneous petrologists
to recover the original tectonic setting of mafic rocks of the
past. When the geological setting alone cannot unambiguously
resolve this question, the chemical composition of
these rocks might contain the answer. The major, minor,
and trace elemental composition of basalts shows large
variations, for example as a function of formation depth
(e.g., Kushiro and Kuno, 1963) --- *Vermeesch (2006)*

For this analysis we are going to use a dataset that was compiled in 

Vermeesch (2006) Tectonic discrimination of basalts with classification trees, *Geochimica et Cosmochimica Acta*  https://doi.org/10.1016/j.gca.2005.12.016

These data were grouped into 3 categories:

- 256 ***Island arc basalts (IAB)*** from the Aeolian, Izu-Bonin, Kermadec, Kurile, Lesser Antilles, Mariana, Scotia, and Tonga arcs.
- 241 ***Mid-ocean ridge (MORB)*** samples from the East Pacific Rise, Mid Atlantic Ridge, Indian Ocean, and Juan de Fuca Ridge.
- 259 ***Ocean-island (OIB)*** samples from St. Helena, the Canary, Cape Verde, Caroline, Crozet, Hawaii-Emperor, Juan Fernandez, Marquesas, Mascarene, Samoan, and Society islands.

**Let's look at the illustration above and determine where each of these settings are within a plate tectonic context**

## Import data


The data are from the supplemental materials of the Vermeesch (2006) paper. The samples are grouped by affinity MORB, OIB, and IAB. They are additionally assigned affinity codes and colors from the default matplotlib cycle:

|affinity| affinity code | color |
|--------|---------------|-------|
| MORB| 0 | C0
| OIB |  1 | C1
| IAB |  2 | C2

In [ ]:
basalt_data = pd.read_csv('./data/Vermeesch2006.csv')
basalt_data.tail()

In [ ]:
MORB_data = basalt_data[basalt_data['affinity']=='MORB']
OIB_data = basalt_data[basalt_data['affinity']=='OIB']
IAB_data = basalt_data[basalt_data['affinity']=='IAB']

## Can geochemical data be used to classify the tectonic setting?

These data are labeled. The author already determined what setting these basalts came from. However, is there are way that we could use these labeled data to determine the setting for an unknown basalt?

A paper published in 1982 proposed that the elements titanium and vanadium were particular good at giving insight into tectonic setting. The details of why are quite complicated and can be summarized as "the depletion of V relative to Ti is a function of the fO2 of the magma and its source, the degree of partial melting, and subsequent fractional crystallization." If you take EPS100B you will learn more about the fundamentals behind this igneous petrology. *For the moment you can consider the working hypothesis behind this classification to that different magmatic environments have differences in oxidation states that are reflected in Ti vs V ratios.*

Shervais, J.W. (1982) Ti-V plots and the petrogenesis of modern and ophiolitic lavas *Earth and Planetary Science Letters* https://doi.org/10.1016/0012-821X(82)90120-0

## Implementing a linear classifier

Rather than using a nearest neighbor approach as we did in class, we could instead implement hard cut offs as lines using a linear classifier. A benefit of using such a classifier is that the data do not need to be normalized between 0 and 1. Instead, the actual values can be used.

In [ ]:
basalt_data_Ti_V = basalt_data[(~basalt_data['TiO2 (wt%)'].isna()) & (~basalt_data['V (ppm)'].isna())]
basalt_data_Ti_V_array = basalt_data_Ti_V[['TiO2 (wt%)','V (ppm)']].values 
basalt_affinity = basalt_data_Ti_V['affinity'].tolist()
basalt_affinity_code = basalt_data_Ti_V['affinity code'].tolist()

In [ ]:
from sklearn.svm import SVC
classifier_svc = SVC(kernel="poly",degree=1)

**Code for you to write: implement the classifier_svc and determine its accuracy using a training set and a test set**

**Code for you to write: pass a grid to the classifier and plot with the data**

We will want to use a grid that is spaced according to the non-normalized data.

In [ ]:
xx, yy = np.meshgrid(np.linspace(0, 5, 101),
                     np.linspace(0, 600, 101))
grid = np.c_[xx.ravel(), yy.ravel()]

## Explore other geochemical parameters of the data and build additional classifiers

**Tasks for you to complete**

- Use the seaborn library and use the sns.pairplot function to make cross-plots of other parameters (https://seaborn.pydata.org/generated/seaborn.pairplot.html)
- *Are there other geochemical parameters that you can use as a classifier that are as good or better than the Ti/V classifier?* Implement another classifier using the classifier type of your choosing (there is that code block in the in-class that shows a lot of them) and determine its accuracy using a training set and a test set to address this question. ***scikit-learn will not be happy with missing values so filter out missing values beforehand***. ***Remember that if you are using the nearest neighbor approach that you need to normalize the data.***
- You can build a classifier that uses more than 2 dimensions. Build a classifier that uses 3 or more parameters instead of 2. When you fit the classifier you provide an array that has:

    `[[data_a_point1,data_b_point1,data_c_point1],[data_a_point2,data_b_point2,data_c_point2]]`

    and then an array of type:

    `[point1_type, point2_type]`
    
     While we had Ti and V in the first array you could have these geochemical data and more so that instead of being 2 x n, it could be 3 x n or 4 x n (where n is the number of data point values).

In [ ]:
import seaborn as sns

**What classifier do you think works best for distinguishing between OIB, MORB and IAB using these geochemical data?**

*Write your answer here*